# SI 650 / EECS 549: Homework 3 Part 3

Homework 3 Part 3 will have you working with deep learning models in a variety of ways. You will likely need to run this on Great Lakes unless you have access to a GPU elsewhere (or be prepared to wait a long time). You should have completed Parts 1 and 2 before attempting this notebook to familiarize yourself with how PyTerrier works.

In Part 3, you'll try the following tasks:
 - Use a large language model to re-rank content
 - Use a text-to-text model to perform query augmentation
 - Train a deep learning IR model and compare its performance.
 
The first two of these tasks will rely on models that we've pretrained for you. However, we've also provided code for how to train these. In the third task, you'll do one simple training and evaluate its results.

For the first two tasks, we've provided most of the code. *You are expected to submit results showing that you have successfully executed it*. You'll need to understand some of the code to complete task 3, which requires you to write new code.

As with the past notebooks, you'll need to have `JAVA_HOME` set, which will need to be run on Great Lakes. You can potentially set it in the notebook with a Jupyter command like
```
!export JAVA_HOME=/fill/in/the/path/to/the/JDK/here
```
by first figuring out where the JDK is installed. (This is setting the `JAVA_HOME` environment variable in the unix way). 

### Install the PyTerrier extensions

You'll need two extensions for [OpenNIR](https://opennir.net/) and [doc2query](https://github.com/terrierteam/pyterrier_doc2query). We've provided the package install commands in comments below.

In [1]:
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git

  Cloning https://github.com/Georgetown-IR-Lab/OpenNIR to /tmp/pip-req-build-s5jlmx0z
  Running command git clone -q https://github.com/Georgetown-IR-Lab/OpenNIR /tmp/pip-req-build-s5jlmx0z
  Cloning https://github.com/terrierteam/pyterrier_doc2query.git to /tmp/pip-req-build-3ol2r6v6
  Running command git clone -q https://github.com/terrierteam/pyterrier_doc2query.git /tmp/pip-req-build-3ol2r6v6


## Getting started

Start PyTerrier as we have in past notebooks.

In [2]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt
import pyterrier_doc2query
import os
import pandas as pd

PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### [TREC-COVID19](https://ir.nist.gov/covidSubmit/) Dataset download

The following cell downloads the [TREC-COVID19](https://ir.nist.gov/covidSubmit/) dataset that we will use periodically throughout this notebook.

In [3]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

# Task 1: Build the inverted index for the TREC-COVID19 collection. (2 points)

Build the index for the TREC Covid-19 (CORD19) data like we have in past notebooks but without any fancy options (e.g., no positional indexing).

In [4]:
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:

    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

## Using an untuned Re-rankers

This notebook will have you work with a few neural re-ranking methods that you've used in class. We can build them from scratch using `onir_pt.reranker` or load them from pretrained models. The models we load from scratch won't have been trained to do IR (yet), however.

And OpenNIR reranking model consists of:
 - `ranker` (e.g., `drmm`, `knrm`, or `pacrr`). This defines the neural ranking architecture. We discussed the `knrm` approach in class.
 - `vocab` (e.g., `wordvec_hash`, or `bert`). This defines how text is encoded by the model. This approach makes it easy to swap out different text representations. 
 
Let's start with the `knrm` method we discussed in class:

In [5]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2021-11-21 18:33:04,820][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2021-11-21 18:33:57,383][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2021-11-21 18:33:57,401][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [51.77s] [682M] [14.4MB/s]
[2021-11-21 18:33:57,416][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [52.60s]
[2021-11-21 18:33:57,416][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2021-11-21 18:34:20,173][WordvecHashVocab][DEBUG] [finished] extracting vecs [22.76s]
[2021-11-21 18:34:20,174][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2021-11-21 18:36:40,917][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:21]
[2021-11-21 18:36:41,053][WordvecHashVocab][DEBUG] [starting] writ

Let's look at how well this model work at ranking compared with our default `BatchRetrieve`

In [6]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[INFO] NumExpr defaulting to 2 threads.


[2021-11-21 18:37:23,222][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 18:37:43,447][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-21 18:37:47,520][onir_pt][DEBUG] [finished] batches: [4.06s] [1250it] [307.62it/s]


,name,map,ndcg,ndcg_cut.10,P.10,mrt
0,DPH,0.068056,0.165653,0.609058,0.658,108.162086
1,DPH >> KNRM,0.054801,0.145450,0.359716,0.450,574.520102


The `knrm` models' performance is lower! The mode doesn't work very well because it hasn't yet been trained for IR; it's using random weights to combine the scores from the similarity matrix--but this is at least a start.

## Loading a trained re-ranker

You can train re-ranking models in PyTerrier using the `fit` method. Here's an example of how to train the `knrm` model on the MS MARCO dataset, which is a large IR collection.

```python
# transfer training signals from a medical sample of MS MARCO
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42) # split into training and validation sets

# Index MS MARCO
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # get top 100 results
            >> pt.text.get_text(train_ds, 'text') # fetch the document text
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # rename columns
            >> knrm) # apply neural re-ranker

pipeline.fit(
    train_topics,
    train_ds.get_qrels(),
    valid_topics,
    train_ds.get_qrels())
```

Training deep learning models takes a bit of time (especially for large datasets like MS MARCO), so we've provided a model that's already been trained for you to download.

In [7]:
del knrm # free up the memory before loading a new version of the ranker (helpful for the GPU)
knrm = onir_pt.reranker.from_checkpoint('http://jurgens.people.si.umich.edu/ir/knrm.medmarco.tar.gz', text_field='abstract', 
                                        expected_md5="d70b1d4f899690dae51161537e69ed5a")

[2021-11-21 18:37:48,233][onir.util.download][DEBUG] downloaded http://jurgens.people.si.umich.edu/ir/knrm.medmarco.tar.gz s] [1.43k] [365kB/s] [md5 hash verified]
[2021-11-21 18:37:48,269][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-11-21 18:38:49,252][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [01:01]


In [8]:
pipeline2 = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline2],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-21 18:39:04,669][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 18:39:04,986][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-21 18:39:08,913][onir_pt][DEBUG] [finished] batches: [3.93s] [1250it] [318.46it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,79.224642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> KNRM,0.065099,0.598,0.160549,0.532602,172.499720,20.0,30.0,0.095533,12.0,26.0,0.024604,20.0,30.0,0.028324,20.0,30.0,0.00597


The tuned performance is a little better than before, but `knrm` still underperforms our first-stage ranking model.

## Reranking with BERT

Large language models such as [BERT](https://arxiv.org/abs/1810.04805) are much more powerful neural models that have been shown to be effective for ranking like we discussed in class. 

Like with `knrm`, we'll start by using BERT for re-ranking with its initial parameters. These parameters have been turned for the masked language modeling (i.e., filling a word in the blank) and predicting the next sentence--but have not been tuned for IR at all.

In [9]:
del knrm # clear out memory from KNRM (useful for GPU)
bert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

[INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpc84mz_kp
100%|██████████| 231508/231508 s<0ms, 320787.87B/s] 
[INFO] copying /tmp/tmpc84mz_kp to cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
[INFO] creating metadata file for /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
[INFO] removing temp file /tmp/tmpc84mz_kp
[INFO] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
[INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert

Let's see how this non-IR trained model does on CORD10 data

In [10]:
pipeline3 = br % 100 >> pt.text.get_text(dataset, 'abstract') >> bert
pt.Experiment(
    [br, pipeline3],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-21 18:40:04,662][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 18:40:04,897][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-21 18:45:51,911][onir_pt][DEBUG] [finished] batches: [05:47] [1250it] [ 3.60it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,55.225842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> VBERT,0.056413,0.458,0.147048,0.374197,7007.943020,8.0,42.0,0.000006,6.0,37.0,9.462198e-07,5.0,45.0,0.000001,8.0,41.0,4.134415e-09


As we see, although the ERT model is pre-trained for recognizing language, it doesn't do very well at ranking on our benchmark. To get better performance, we'll need to tune for the task of relevance ranking.

We can train the model for ranking (as shown above for KNRM) or we can download a trained model. Here, we will use the [SLEDGE](https://arxiv.org/abs/2010.05987) model, which is a BERT model trained on scientific text and tuned on medical queries.

In [11]:
bert = onir_pt.reranker.from_checkpoint('http://jurgens.people.si.umich.edu/ir/scibert-medmarco.tar.gz', 
                                         text_field='abstract', expected_md5="854966d0b61543ffffa44cea627ab63b")

[2021-11-21 18:48:45,165][onir.util.download][DEBUG] downloaded http://jurgens.people.si.umich.edu/ir/scibert-medmarco.tar.gz [02:52] [499M] [4.36MB/s] [md5 hash verified]


[2021-11-21 18:49:10,469][onir.util.download][DEBUG] downloaded https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar [20.22s] [411M] [22.8MB/s] [md5 hash verified]


extracting: 411MB [1.53s, 268MB/s]
extracting: 821MB [9.14s, 89.7MB/s]
[INFO] Model name '/root/data/onir/bert_models/scibert-scivocab-uncased' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/root/data/onir/bert_models/scibert-scivocab-uncased' is a path or url to a directory containing tokenizer files.
[INFO] Didn't find file /root/data/onir/bert_models/scibert-scivocab-uncased/added_tokens.json. We won't load it.
[INFO] Didn't find file /root/data/onir/bert_models/scibert-scivocab-uncased/special_tokens_map.json. We won't load it.
[INFO] loading file /root/data/onir/bert_models/scibert-scivoca

Let's run another experiment to see how this new model trained for IR does.

In [12]:
pipeline4 = br % 100 >> pt.text.get_text(dataset, 'abstract') >> bert
pt.Experiment(
    [br, pipeline4],
    topics,
    qrels,
    names=['DPH', 'DPH >> Trained-BERT'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-21 18:49:38,898][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 18:49:39,123][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-21 18:55:04,921][onir_pt][DEBUG] [finished] batches: [05:26] [1250it] [ 3.84it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,DPH,0.068056,0.658,0.165653,0.609058,55.703740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPH >> Trained-BERT,0.075710,0.770,0.173079,0.701995,6580.968283,36.0,14.0,0.001278,28.0,11.0,0.000851,36.0,14.0,0.010118,31.0,19.0,0.012156


Training helped a lot! We're able to improve upon the initial ranking from `BatchRetrieve`. However, from looking at `mrt` we can see that this is pretty slow to run--and this was using a GPU! This performance time underscores the trade-off in using large language models at retrieval time: they may perform better, but could be much slower.

# Deep learning at indexing time: doc2query

Instead of using our large language models to rerank, another option is to use them at _indexing time_ to augment our documents. In class, we discussed one such option, doc2query, that augments an inverted index structure by predicting queries that may be used to search for the document, and appending those to the document text.

We can use doc2query using the `pyterrier_doc2query` package, which was loaded at the top.

### Loading a pre-trained model

We'll start by using a version of the doc2query model released by the authors that is trained on the MS MARCO collection.

In [13]:
if not os.path.exists("t5-base.zip"):
  !wget http://jurgens.people.si.umich.edu/ir/t5-base.zip
  !unzip t5-base.zip

--2021-11-21 18:55:05--  http://jurgens.people.si.umich.edu/ir/t5-base.zip
Resolving jurgens.people.si.umich.edu (jurgens.people.si.umich.edu)... 141.211.184.98
Connecting to jurgens.people.si.umich.edu (jurgens.people.si.umich.edu)|141.211.184.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357139559 (341M) [application/zip]
Saving to: ‘t5-base.zip’

t5-base.zip         100%[===================>] 340.59M  7.08MB/s    in 94s     

2021-11-21 18:56:41 (3.61 MB/s) - ‘t5-base.zip’ saved [357139559/357139559]

Archive:  t5-base.zip
  inflating: model.ckpt-1004000.data-00000-of-00002  
  inflating: model.ckpt-1004000.data-00001-of-00002  
  inflating: model.ckpt-1004000.index  
  inflating: model.ckpt-1004000.meta  


We can load the model weights by specifying the checkpoint.

In [14]:
doc2query = pyterrier_doc2query.Doc2Query('model.ckpt-1004000', batch_size=8)

Downloading:   0%|          | 0.00/773k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M s<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k s<?, ?B/s]

Doc2query using cuda


### Running doc2queries on sample text

Let's see what queries it predicts for the sample document that we've made up:

In [15]:
df = pd.DataFrame([{"docno" : "d1", "text" :"The University of Michigan School of Information (UMSI) delivers innovative, elegant and ethical solutions connecting people, information and technology. The school was one of the first iSchools in the nation and is the premier institution studying and using technology to improve human computer interactions."}])
df.iloc[0].text

'The University of Michigan School of Information (UMSI) delivers innovative, elegant and ethical solutions connecting people, information and technology. The school was one of the first iSchools in the nation and is the premier institution studying and using technology to improve human computer interactions.'

In [16]:
doc2query_df = doc2query(df)
doc2query_df.iloc[0].querygen

'what is umsi university of michigan school of information what is umsi'

Not too bad, though the questions are somewhat generic

### Loading an index of doc2query documents

Let's see how doc2query does on improving the performance in the TREC COVID data. Since indexing with doc2query takes a while (due to needing to run the deep learning models), we've provided an index with the text already added.

If you would like to index the collection with doc2query yourself (or use doc2query for your course project), you can use the following code:

```python
dataset = pt.get_dataset("irds:cord19/trec-covid")
indexer = (
  pyterrier_doc2query.Doc2Query('model.ckpt-1004000', doc_attr='abstract', batch_size=8, append=True) # aply doc2query on abstracts and append
  >> pt.apply.generic(lambda df: df.rename(columns={'abstract': 'text'}) # rename "abstract" column to "text" for indexing
  >> pt.IterDictIndexer("./doc2query_index_path")) # index the expanded documents
indexref = indexer.index(dataset.get_corpus_iter())
```


In [17]:
if not os.path.exists('doc2query_marco_cord19.zip'):
  !wget http://jurgens.people.si.umich.edu/ir/doc2query_marco_cord19.zip
  !unzip doc2query_marco_cord19.zip
doc2query_indexref = pt.IndexRef.of('./doc2query_index_path/data.properties')

--2021-11-21 18:57:06--  http://jurgens.people.si.umich.edu/ir/doc2query_marco_cord19.zip
Resolving jurgens.people.si.umich.edu (jurgens.people.si.umich.edu)... 141.211.184.98
Connecting to jurgens.people.si.umich.edu (jurgens.people.si.umich.edu)|141.211.184.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45804576 (44M) [application/zip]
Saving to: ‘doc2query_marco_cord19.zip’

doc2query_marco_cor 100%[===================>]  43.68M  5.00MB/s    in 8.8s    

2021-11-21 18:57:15 (4.97 MB/s) - ‘doc2query_marco_cord19.zip’ saved [45804576/45804576]

Archive:  doc2query_marco_cord19.zip
   creating: doc2query_index_path/
  inflating: doc2query_index_path/data.document.fsarrayfile  
  inflating: doc2query_index_path/data.inverted.bf  
  inflating: doc2query_index_path/data.direct.bf  
  inflating: doc2query_index_path/data.lexicon.fsomapid  
  inflating: doc2query_index_path/data.lexicon.fsomaphash  
  inflating: doc2query_index_path/data.lexicon.fsomapfile  
  i

Let's look at the results on TREC COVID by first merging the scores with the rankings

In [18]:
dataset = pt.get_dataset('irds:cord19/trec-covid')
pipeline = pt.BatchRetrieve(doc2query_indexref) % 1 >> pt.text.get_text(dataset, 'title')
res = pipeline(dataset.get_topics('title'))
res.merge(dataset.get_qrels(), how='left').head()

,qid,docid,docno,rank,score,query,title,label,iteration
0,1,101299,jwmrgy5d,0,8.427298,coronavirus origin,COVID-19 in the heart and the lungs: could we ...,0.0,5
1,2,182167,g8grcy5j,0,13.922648,coronavirus response to weather changes,The Stirling Protocol – Putting the environmen...,0.0,4
2,3,85678,tl30wlpy,0,7.224180,coronavirus immunity,Receptor-dependent coronavirus infection of de...,NaN,NaN
3,4,145871,l5fxswfz,0,12.773362,how do people die from the coronavirus,Analysis on 54 Mortality Cases of Coronavirus ...,2.0,1.5
4,5,180990,3sepefqa,0,12.995980,animal models of covid 19,Current global vaccine and drug efforts agains...,0.0,4


What kind of queries does doc2query generate for the CORD19 documents?

In [19]:
df = pd.DataFrame(doc for doc in dataset.get_corpus_iter() if doc['docno'] in ('3sepefqa', 'l5fxswfz'))
df = df.rename(columns={'abstract': 'text'})
doc2query_df = doc2query(df)
for querygen, docno, text in zip(doc2query_df['querygen'], doc2query_df['docno'], df['text']):
    print(docno)
    print(querygen)
    print(text)

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

l5fxswfz
what is the current number of cases of coronavirus disease worldwide? how many coronavirus cases in the world how many coronaviruses are there
Since the identification of the first case of coronavirus disease 2019 (COVID-19), the global number of confirmed cases as of March 15, 2020, is 156,400, with total death in 5,833 (3.7%) worldwide. Here, we summarize the morality data from February 19 when the first mortality occurred to 0 am, March 10, 2020, in Korea with comparison to other countries. The overall case fatality rate of COVID-19 in Korea was 0.7% as of 0 am, March 10, 2020.
3sepefqa
what is the medicine used for copid what is copid 19 medication what is the cure for the comid epidemic
COVID-19 has become one of the biggest health concern, along with huge economic burden. With no clear remedies to treat the disease, doctors are repurposing drugs like chloroquine and remdesivir to treat COVID-19 patients. In parallel, research institutes in collaboration with biotech comp

## Evaluating the effects of doc2query

Here, we'll change our evaluation setup a bit from what we did before. Rather than compare two models for the same index, we'll instead compare the same model (BM25) with two different ways of indexing (two indices)! Our baseline will be an index of CORD19 without the doc2query additions.

Let's load a copy of the CORD19 index that we used earlier.

In [5]:
indexref = pt.IndexRef.of('./terrier_cord19/data.properties')

### Task 2: Write the `Experiment` to compare indices (3 points)
Run an `Experiment` using a `BM25` ranker that compares the indices `indexref` and `doc2query_indexref`. Compare your models using MAP, NDCG, and NDCG@10. Note that our doc2query model was trained on MS MARCO, which isn't the same kind of collection as CORD19, so this performance tells us how well that model can transfer to a new setting.

In [21]:
pt.Experiment([
  pt.BatchRetrieve(indexref, wmodel="BM25"),
  pt.BatchRetrieve(doc2query_indexref, wmodel="BM25"),
  ],
  topics,
  qrels,
  baseline=0,
  names=["BM25", "BM25_doc2query"],
  eval_metrics=["map", "ndcg", "ndcg_cut.10"]
)

,name,map,ndcg,ndcg_cut.10,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,BM25,0.195498,0.411103,0.644463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25_doc2query,0.196293,0.412673,0.623652,30.0,20.0,0.704711,30.0,20.0,0.530788,20.0,27.0,0.191547


# Task 3: Train a new model! (25 points)

All of the prior exercises have had you working with either off-the-shelf models (not trained for IR) or models that someone else has trained for you. To give you a sense of how to train a model, your primary task in this notebook is to train a simple `knrm` model, which should be relatively efficient to train on a GPU. 

To keep things simple, we'll use the same setup for CORD19 that we did in Part 2 (30 queries in train, 5 in dev, 15 queries in test) which is still relatively small for training a deep learning model but will get you started on the process. 

Your tasks are the following:
- Load the CORD19 dataset and split it into train, dev, and test
- Create a new `knrm` ranker and a pipeline that uses it
- Run an `Experiment` comparing four models:
  - a default `BatchRetrieve`, filtering to the top 100 results
  - BM25, filtering to the top 100 results
  - a pipeline that feeds the top 100 results of the default `BatchRetrieve` to your `knrm` model
  - a pipeline that feeds the top 100 results of BM25 to your `knrm` model
  
Your `Experiment` should evaluate models using MAP, NDCG, NDCG@10, Precision@10, and Mean Response Time.
  
We expect to see the `Experiment`'s results in the final cell. You are, of course, welcome to try training any of the fancier models to see how they do as well!

In [6]:
RANK_CUTOFF = 10
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

In [7]:
del bert # free up the memory before loading a new version of the ranker (helpful for the GPU)
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

pipeline = (pt.BatchRetrieve(indexref) % 100 # get top 100 results
            >> pt.text.get_text(dataset, 'abstract') # fetch the document text
            >> knrm) # apply neural re-ranker

pipeline.fit(
    train_topics,
    dataset.get_qrels(),
    valid_topics,
    dataset.get_qrels())

config file not found: config
[2021-11-21 19:21:08,816][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-11-21 19:22:09,723][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [01:01]
[2021-11-21 19:22:20,388][onir_pt][DEBUG] [starting] validation
[2021-11-21 19:22:20,453][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:22:40,520][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 19:22:41,305][onir_pt][DEBUG] [finished] batches: s] [125it] [159.69it/s]
[2021-11-21 19:22:41,592][onir_pt][DEBUG] [finished] validation [21.20s]
[2021-11-21 19:22:41,594][onir_pt][INFO] pre-validation: 0.0073
[2021-11-21 19:22:41,616][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:22:41,617][onir_pt][DEBUG] [starting] training
[2021-11-21 19:22:41,618][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 19:22:50,676][onir_pt][DEBUG] [finished] train pairs: [9.06s] [1024it] [113.05it/s]
[2021-11-21 19:22:50,682][onir_pt][DEBUG] [finished] training [9.06s]
[2021-11-21 19:22:50,683][onir_pt][INFO] training   it=0 loss=0.2680
[2021-11-21 19:22:50,684][onir_pt][DEBUG] [starting] validation
[2021-11-21 19:22:50,685][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:22:50,691][onir_pt][DEBUG] [starting] batches


...

[2021-11-21 19:32:40,685][onir_pt][DEBUG] [finished] batches: s] [125it] [224.02it/s]
[2021-11-21 19:32:40,965][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 19:32:40,967][onir_pt][INFO] validation it=53 map=0.0089 ndcg=0.0209 P_10=0.0740
[2021-11-21 19:32:40,968][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:32:40,969][onir_pt][DEBUG] [starting] training
[2021-11-21 19:32:40,970][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 19:32:52,282][onir_pt][DEBUG] [finished] train pairs: [11.31s] [1024it] [90.53it/s]
[2021-11-21 19:32:52,287][onir_pt][DEBUG] [finished] training [11.32s]
[2021-11-21 19:32:52,287][onir_pt][INFO] training   it=54 loss=0.1785
[2021-11-21 19:32:52,288][onir_pt][DEBUG] [starting] validation
[2021-11-21 19:32:52,288][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:32:52,291][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 19:32:52,815][onir_pt][DEBUG] [finished] batches: s] [125it] [243.94it/s]
[2021-11-21 19:32:53,085][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 19:32:53,086][onir_pt][INFO] validation it=54 map=0.0088 ndcg=0.0205 P_10=0.0700
[2021-11-21 19:32:53,086][onir_pt][INFO] early stopping; model reverting back to it=34


In [8]:
del knrm # free up the memory before loading a new version of the ranker (helpful for the GPU)
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

pipeline2 = (pt.BatchRetrieve(indexref, wmodel="BM25") % 100 # get top 100 results
            >> pt.text.get_text(dataset, 'abstract') # fetch the document text
            >> knrm) # apply neural re-ranker

pipeline2.fit(
    train_topics,
    dataset.get_qrels(),
    valid_topics,
    dataset.get_qrels())

[2021-11-21 19:35:10,030][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2021-11-21 19:36:10,188][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [01:00]
[2021-11-21 19:36:18,637][onir_pt][DEBUG] [starting] validation
[2021-11-21 19:36:18,639][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:36:18,878][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 19:36:19,321][onir_pt][DEBUG] [finished] batches: s] [125it] [283.15it/s]
[2021-11-21 19:36:19,504][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 19:36:19,505][onir_pt][INFO] pre-validation: 0.0085
[2021-11-21 19:36:19,513][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:36:19,515][onir_pt][DEBUG] [starting] training
[2021-11-21 19:36:19,516][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 19:36:28,016][onir_pt][DEBUG] [finished] train pairs: [8.50s] [1024it] [120.48it/s]
[2021-11-21 19:36:28,024][onir_pt][DEBUG] [finished] training [8.51s]
[2021-11-21 19:36:28,025][onir_pt][INFO] training   it=0 loss=0.2460
[2021-11-21 19:36:28,026][onir_pt][DEBUG] [starting] validation
[2021-11-21 19:36:28,028][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:36:28,034][onir_pt][DEBUG] [starting] batches


...

[2021-11-21 19:42:27,716][onir_pt][DEBUG] [finished] batches: s] [125it] [233.72it/s]
[2021-11-21 19:42:27,931][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 19:42:27,934][onir_pt][INFO] validation it=34 map=0.0105 ndcg=0.0231 P_10=0.0720
[2021-11-21 19:42:27,935][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:42:27,937][onir_pt][DEBUG] [starting] training
[2021-11-21 19:42:27,938][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2021-11-21 19:42:38,820][onir_pt][DEBUG] [finished] train pairs: [10.88s] [1024it] [94.12it/s]
[2021-11-21 19:42:38,824][onir_pt][DEBUG] [finished] training [10.89s]
[2021-11-21 19:42:38,825][onir_pt][INFO] training   it=35 loss=0.1818
[2021-11-21 19:42:38,825][onir_pt][DEBUG] [starting] validation
[2021-11-21 19:42:38,826][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:42:38,827][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/125 s<?, ?it/s]

[2021-11-21 19:42:39,390][onir_pt][DEBUG] [finished] batches: s] [125it] [222.43it/s]
[2021-11-21 19:42:39,577][onir_pt][DEBUG] [finished] validation s]
[2021-11-21 19:42:39,578][onir_pt][INFO] validation it=35 map=0.0104 ndcg=0.0229 P_10=0.0700
[2021-11-21 19:42:39,578][onir_pt][INFO] early stopping; model reverting back to it=15


In [9]:
br = pt.BatchRetrieve(index)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [10]:
pt.Experiment(
    [br, bm25, pipeline, pipeline2],
    topics,
    qrels,
    names=['BR', 'BM25', 'BR >> KNRM', 'BM25 >> KNRM'],
    baseline=0,
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

[2021-11-21 19:50:35,649][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:50:35,651][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-21 19:50:39,455][onir_pt][DEBUG] [finished] batches: [3.80s] [1250it] [328.71it/s]
[2021-11-21 19:50:42,338][onir_pt][DEBUG] using GPU (deterministic)
[2021-11-21 19:50:42,340][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2021-11-21 19:50:46,119][onir_pt][DEBUG] [finished] batches: [3.78s] [1250it] [330.88it/s]


,name,map,P.10,ndcg,ndcg_cut.10,mrt,map +,map -,map p-value,P.10 +,P.10 -,P.10 p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,BR,0.170652,0.658,0.380035,0.609058,62.522009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25,0.195498,0.692,0.411103,0.644463,56.767937,35.0,15.0,6.542654e-05,18.0,13.0,0.126526,39.0,11.0,2.975186e-05,25.0,21.0,0.050516
2,BR >> KNRM,0.067161,0.630,0.163482,0.573155,147.344117,0.0,50.0,3.265386e-10,16.0,21.0,0.345930,0.0,50.0,2.391228e-19,25.0,24.0,0.227067
3,BM25 >> KNRM,0.075084,0.650,0.172662,0.572015,128.921196,2.0,48.0,1.092793e-08,19.0,19.0,0.789527,0.0,50.0,1.961271e-17,24.0,26.0,0.230174
